In [11]:
# numpy,csv모듈 불러옴
#난수 발생 패턴을 고정, 하이퍼파라미터값 선언한 셈
%run ../chap01/abalone.ipynb

#메인 함수 정의
def pulsar_exec(epoch_count=10, mb_size=10, report=1):
    load_pulsar_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)
    
#데이터 적재 함수 정의
def load_pulsar_dataset():
    #csv 패키지의 기능을 이요하여 펄서 예측 데이터셋 파일 내용을 메모리로 읽음
    with open("C:/Users/KimSeongHoon/D_test/chap02/pulsar_stars.csv") as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        rows = []
        #리스트 변수 rows에 저장된 내용을 배열 변수 data에 복사
        for row in csvreader:
            rows.append(row)
            
    global data, input_cnt, output_cnt 
    input_cnt, output_cnt = 8, 1 #입출력 벡터의 크기는 8과 1
    data = np.asarray(rows, dtype='float32') #리스트 구조를 배열 구조로 변환하는 과정
    
#후퍼리 과정에 대한 순전파와 역전파 함수의 재정의
def forward_postproc(output, y):
    entropy = sigmoid_cross_entropy_with_logits(y, output) #두 행렬의 원소 짝에 대한 시그모이드 교차 엔트로피값을 구하는 두 단계 연산 수행
    loss = np.mean(entropy) # 위 계산의 평균으로 loss값 구하기
    return loss, [y, output, entropy]

def backprop_postproc(G_loss, aux):
    y, output, entropy = aux
    
    g_loss_entropy = 1.0 / np.prod(entropy.shape) # 1.0을 행렬의 원소 수로 나누어 각 원소의 손실 기울기로 부여
    g_entropy_output = sigmoid_cross_entropy_with_logits_derv(y, output)   
    
    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy
    
    return G_output

#정확도 계산 함수의 재정의
def eval_accuracy(output, y):
    estimate = np.greater(output, 0)
    answer = np.greater(y, 0.5)
    correct = np.equal(estimate, answer)
    
    return np.mean(correct)

#시그모이드 관련 함수 정의
def relu(x):
    return np.maximum(x, 0)

def sigmoid(x):
    return np.exp(-relu(-x)) / (1.0 + np.exp(-np.abs(x)))
        
def sigmoid_derv(x, y):
    return y * (1 - y)

def sigmoid_cross_entropy_with_logits(z, x):
    return relu(x) - x * z + np.log(1 + np.exp(-np.abs(x)))

def sigmoid_cross_entropy_with_logits_derv(z, x):
    return -z + sigmoid(x)